In [1]:
import os
from pathlib import Path

cwd = os.getcwd()
if cwd.split("/")[-1] == "notebooks":
    cwd = Path(cwd).parent
    os.chdir(cwd)
    print("Current working directory:", os.getcwd())

Current working directory: /Users/samlupton/Desktop/ML Projects/compromeets


# Map isochrones (skip)

In [24]:
import datetime
from pathlib import Path

import pandas as pd
import r5py
import shapely

# Load transport network
gtfs_path = Path("compromeets/artifacts/london_transport_gtfs.zip")
osm_path = Path("compromeets/artifacts/greater-london-260121.osm.pbf")

transport_network = r5py.TransportNetwork(
    osm_pbf=osm_path,
    gtfs=[
        gtfs_path,
    ],
)

# Create isochrone from a point
origin = shapely.Point(-0.1276, 51.5074)  # London coordinates

isochrones = r5py.Isochrones(
    transport_network=transport_network,
    origins=origin,
    departure=datetime.datetime(2026, 2, 1, 8, 0),  # Must match GTFS service times - using a date with active service
    transport_modes=[r5py.TransportMode.TRANSIT, r5py.TransportMode.WALK],
    isochrones=[15, 30, 45],
)
print(isochrones)

isochrones_cleaned = isochrones.copy()
isochrones_cleaned["travel_time"] = (isochrones_cleaned["travel_time"].dt.total_seconds() / 60).astype(int)
isochrones_cleaned.explore(column="travel_time", cmap="YlOrRd")

      travel_time                                           geometry
0 0 days 00:15:00  MULTILINESTRING ((-0.09501 51.51255, -0.09501 ...
1 0 days 00:30:00  MULTILINESTRING ((0.00575 51.49979, 0.00575 51...
2 0 days 00:45:00  MULTILINESTRING ((0.04853 51.53863, 0.04899 51...


# Load postcode data/transport networks and calculate travel time between 2 points
This will be used to calculate the size of the isochrone needed

In [7]:
import datetime
from itertools import combinations
from pathlib import Path

import folium
import geopandas as gpd
import pandas as pd
import r5py
import shapely
from shapely.geometry import Point
from shapely.ops import unary_union

# Load transport network
gtfs_path = Path("compromeets/artifacts/london_transport_gtfs.zip")
osm_path = Path("compromeets/artifacts/greater-london-260121.osm.pbf")

transport_network = r5py.TransportNetwork(
    osm_pbf=osm_path,
    gtfs=[
        gtfs_path,
    ],
)

postcodes_gdf = pd.read_csv("~/Downloads/ONSPD_NOV_2025/Data/ONSPD_NOV_2025_UK.csv")[["pcds", "lat", "long"]]
postcodes_gdf = gpd.GeoDataFrame(postcodes_gdf, geometry=gpd.points_from_xy(postcodes_gdf.long, postcodes_gdf.lat))


def postcode_to_point(postcode, postcodes_gdf):
    return postcodes_gdf[postcodes_gdf.pcds == postcode].geometry.values[0]


def travel_time_between_postcodes(postcodes: list[str], postcodes_gdf: gpd.GeoDataFrame, transport_network: r5py.TransportNetwork):
    postcode_points = [postcode_to_point(postcode, postcodes_gdf) for postcode in set(postcodes)]
    num_points = len(postcode_points)
    origins = gpd.GeoDataFrame({"id": [f"origin{i}" for i in range(num_points)]}, geometry=postcode_points, crs="EPSG:4326")
    destinations = gpd.GeoDataFrame({"id": [f"dest{i}" for i in range(num_points)]}, geometry=postcode_points, crs="EPSG:4326")
    # Calculate travel time matrix (fastest method)
    travel_times = r5py.TravelTimeMatrix(
        transport_network=transport_network,
        origins=origins,
        destinations=destinations,
        departure=datetime.datetime(2026, 2, 1, 8, 0),
        transport_modes=[r5py.TransportMode.TRANSIT, r5py.TransportMode.WALK],
    )

    return int(max(travel_times["travel_time"]))

/var/folders/vv/2byc96_n12g2d0tqxrl3s8sw0000gn/T/ipykernel_80450/2348326598.py:24: DtypeWarning: Columns (18,31,39,44,52) have mixed types. Specify dtype option on import or set low_memory=False.
  postcodes_gdf = pd.read_csv("~/Downloads/ONSPD_NOV_2025/Data/ONSPD_NOV_2025_UK.csv")[["pcds", "lat", "long"]]


# Calculate Isochrones

In [8]:
isochrone_travel_time = travel_time_between_postcodes(["E14 2DF", "N7 9RA", "SW2 1AB"], postcodes_gdf, transport_network) * 0.7

In [12]:
def postcode_to_isochrones(
    postcode,
    postcodes_gdf,
    transport_network,
    departure_time=datetime.datetime(2026, 2, 1, 8, 0),
    travel_times=[15, 30, 45],
):
    origin = postcode_to_point(postcode, postcodes_gdf)

    isochrones = r5py.Isochrones(
        transport_network=transport_network,
        origins=origin,
        departure=departure_time,  # Must match GTFS service times - using a date with active service
        transport_modes=[r5py.TransportMode.TRANSIT, r5py.TransportMode.WALK],
        isochrones=travel_times,
    )

    return isochrones, origin


postcodes = set(["E14 2DF", "N7 9RA", "SW2 1AB"])
isochrone_travel_time = travel_time_between_postcodes(postcodes, postcodes_gdf, transport_network) * 0.7

isochrones_origins = [postcode_to_isochrones(
    postcode, postcodes_gdf, transport_network, travel_times=[isochrone_travel_time]
) for postcode in postcodes]

# Calculate and Map Overlap

In [17]:
def find_meeting_places(
    people_data: list[tuple[gpd.GeoDataFrame, shapely.Point]],
    travel_time_minutes: int,
    postcodes_gdf: gpd.GeoDataFrame,
    map_center: tuple[float, float] = (51.5074, -0.1276),
    map_zoom: int = 11,
):
    """
    Find meeting places accessible to all people within the specified travel time.
    
    Args:
        people_data: List of tuples, each containing (isochrones_gdf, origin_point)
        travel_time_minutes: Travel time in minutes to extract from isochrones
        postcodes_gdf: GeoDataFrame containing postcode data
        map_center: Center point for the map (lat, lng)
        map_zoom: Initial zoom level for the map
    
    Returns:
        tuple: (overlap_polygon, overlap_gdf, postcodes_in_overlap, map_object)
    """
    import matplotlib.colors as mcolors
    
    def linestring_to_polygon(gdf):
        """Convert MULTILINESTRING isochrones to polygons"""
        unioned = unary_union(gdf.geometry.values)
        if unioned.geom_type == "Polygon":
            return unioned
        elif unioned.geom_type == "MultiLineString":
            buffered = unioned.buffer(0.0001)
            return buffered.convex_hull
        else:
            return unioned.convex_hull
    
    # Extract isochrones for each person and convert to polygons
    polygons = []
    origins = []
    
    for i, (isochrones_gdf, origin_point) in enumerate(people_data):
        # Extract the isochrone for the specified travel time
        iso = isochrones_gdf[isochrones_gdf["travel_time"] == pd.Timedelta(minutes=travel_time_minutes)]
        if len(iso) == 0:
            print(f"Warning: No isochrone found for person {i+1} at {travel_time_minutes} minutes")
            continue
        
        # Convert to polygon
        poly = linestring_to_polygon(iso)
        polygons.append(poly)
        origins.append((i+1, origin_point))
    
    if len(polygons) == 0:
        raise ValueError("No valid isochrones found for any person")
    
    # Find the intersection of all polygons
    if len(polygons) == 1:
        overlap = polygons[0]
    else:
        overlap = polygons[0]
        for poly in polygons[1:]:
            overlap = overlap.intersection(poly)
    
    # Create a GeoDataFrame for the overlap
    overlap_gdf = gpd.GeoDataFrame({"description": ["Overlap area"]}, geometry=[overlap], crs="EPSG:4326")
    
    print(f"Overlap area: {overlap.area * 111000 * 111000:.2f} square meters")
    print(f"Overlap exists: {not overlap.is_empty}")
    
    # Find postcodes within the overlap
    if not overlap.is_empty:
        postcodes_in_overlap = postcodes_gdf[postcodes_gdf.geometry.within(overlap_gdf.geometry.iloc[0])]
    else:
        postcodes_in_overlap = gpd.GeoDataFrame(columns=postcodes_gdf.columns)
    
    print(f"\nNumber of postcodes in overlap: {len(postcodes_in_overlap)}")
    if len(postcodes_in_overlap) > 0:
        print(postcodes_in_overlap[["pcds", "lat", "long"]].head())
    
    # Generate colors for each person
    colors = ['blue', 'green', 'red', 'purple', 'orange', 'darkred', 'lightred', 'beige', 
              'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 
              'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']
    
    # Visualize on a map
    m = folium.Map(location=map_center, zoom_start=map_zoom, tiles="OpenStreetMap")
    
    # Add isochrones for each person
    for i, poly in enumerate(polygons):
        color = colors[i % len(colors)]
        folium.GeoJson(
            poly,
            style_function=lambda feature, c=color: {
                "fillColor": c,
                "color": c,
                "weight": 2,
                "fillOpacity": 0.3,
            },
            tooltip=f"Person {i+1} - {travel_time_minutes} min",
        ).add_to(m)
    
    # Add overlap area
    if not overlap.is_empty:
        folium.GeoJson(
            overlap,
            style_function=lambda feature: {
                "fillColor": "red",
                "color": "red",
                "weight": 3,
                "fillOpacity": 0.5,
            },
            tooltip=f"Overlap - Accessible to all {len(people_data)} people",
        ).add_to(m)
    
    # Add origin markers
    for person_num, origin in origins:
        color = colors[(person_num - 1) % len(colors)]
        folium.Marker(
            [origin.y, origin.x],
            popup=f"Person {person_num}",
            icon=folium.Icon(color=color, icon="user")
        ).add_to(m)
    
    # Optionally add postcodes in overlap
    if len(postcodes_in_overlap) > 0:
        for _, row in postcodes_in_overlap.head(50).iterrows():
            folium.CircleMarker(
                [row["lat"], row["long"]],
                radius=3,
                popup=row["pcds"],
                color="red",
                fill=True,
                fillColor="red"
            ).add_to(m)
    
    return overlap, overlap_gdf, postcodes_in_overlap, m

# Call the function
overlap, overlap_gdf, postcodes_in_overlap, map_obj = find_meeting_places(
    people_data=isochrones_origins,
    travel_time_minutes=isochrone_travel_time,
    postcodes_gdf=postcodes_gdf,
)

# Display the map
map_obj

Overlap area: 21882868.20 square meters
Overlap exists: True

Number of postcodes in overlap: 22395
            pcds        lat      long
777292  EC1A 1AA  51.525138 -0.111157
777293  EC1A 1AE  51.515212 -0.099043
777294  EC1A 1AH  51.524567 -0.112017
777295  EC1A 1AL  51.517884 -0.097490
777296  EC1A 1AR  51.524505 -0.112063


# Search in Google Places API

In [18]:
centroid = overlap.centroid
center_lat = centroid.y
center_lng = centroid.x

# Calculate approximate radius in meters
# Get the distance from center to farthest point

max_distance = (
    max([Point(center_lng, center_lat).distance(Point(coord[0], coord[1])) for coord in overlap.exterior.coords])
    * 111000
)  # Rough conversion to meters (1 degree ≈ 111km)

# For Google Maps Nearby Search
location = {"latitude": center_lat, "longitude": center_lng}
radius = int(max_distance)

In [19]:
location

{'latitude': 51.51009997860904, 'longitude': -0.10580316628407935}

In [21]:
import os
from datetime import datetime

import googlemaps
import requests

API_KEY = os.getenv("GOOGLE_PLACES_API_KEY")


gmaps = googlemaps.Client(key=API_KEY)

url = "https://places.googleapis.com/v1/places:searchNearby"
headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": API_KEY,
    "X-Goog-FieldMask": "places.displayName,places.rating,places.userRatingCount,places.location",
}
# Place types can be found here:
# https://developers.google.com/maps/documentation/places/web-service/place-types#table-a
json_data = {
    "includedTypes": ["pub"],
    "maxResultCount": 10,
    "locationRestriction": {
        "circle": {"center": {"latitude": location["latitude"], "longitude": location["longitude"]}, "radius": 500.0}
    },
}

request = requests.post(url, json=json_data, headers=headers)

In [23]:
import os
from datetime import datetime

import googlemaps
import requests

API_KEY = os.getenv("GOOGLE_PLACES_API_KEY")

# Use searchText endpoint for better text-based filtering
url = "https://places.googleapis.com/v1/places:searchText"
headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": API_KEY,
    "X-Goog-FieldMask": "places.displayName,places.rating,places.userRatingCount,places.location,places.types",
}

json_data = {
    "textQuery": "pubs showing sports",  # Text query for sports pubs
    "includedType": "pub",  # Filter to pubs only
    "maxResultCount": 20,
    "locationBias": {
        "circle": {
            "center": {"latitude": location["latitude"], "longitude": location["longitude"]},
            "radius": radius
        }
    },
}

request = requests.post(url, json=json_data, headers=headers)
results = request.json()

In [ ]:
sorted_places = sorted(
    [(place["displayName"]["text"], place.get("rating", 0)) for place in results["places"]],
    key=lambda x: x[1],
    reverse=True
)
sorted_places

[('Goldwood', 4.8),
 ('Redwood', 4.6),
 ('The Court', 4.6),
 ('Beechwood', 4.6),
 ('Kings Arms', 4.5),
 ('The Three Lanterns', 4.4),
 ('One Tun', 4.4),
 ('The Northumberland Arms', 4.4),
 ('Fox on the Green', 4.4),
 ('The Albion', 4.3),
 ('Greenwood', 4.3),
 ("The Pavilion End, St Paul's", 4.3),
 ('The Old Kings Head', 4.3),
 ('Golden Lion', 4.2),
 ('The Green Man', 4.2),
 ('The Hole In The Wall', 4.1),
 ('The Thirsty Bear', 4.1),
 ('The Bridge Tap, London', 4.1),
 ('The Flying Horse', 4.1),
 ('The Coach House', 4.1)]

In [ ]:
import os

API_KEY = os.getenv("GOOGLE_PLACES_API_KEY")
location = {"latitude": 37.7937, "longitude": -122.3965}

In [7]:
os.getenv("GOOGLE_PLACES_API_KEY")

In [9]:
from compromeets.google_places_client import GooglePlacesClient

with GooglePlacesClient(API_KEY) as client:
    response = client.search_nearby(location, 500, ["pub"])

{'places': [{'location': {'latitude': 37.794061299999996, 'longitude': -122.401694}, 'rating': 4.3, 'userRatingCount': 1110, 'displayName': {'text': 'Irish Times', 'languageCode': 'en'}}, {'location': {'latitude': 37.7941431, 'longitude': -122.3992091}, 'rating': 4.3, 'userRatingCount': 741, 'displayName': {'text': 'Royal Exchange Restaurant', 'languageCode': 'en'}}, {'location': {'latitude': 37.794027199999995, 'longitude': -122.399087}, 'rating': 4, 'userRatingCount': 482, 'displayName': {'text': "Harrington's Bar and Grill - Irish Pub", 'languageCode': 'en'}}, {'location': {'latitude': 37.795, 'longitude': -122.39888889999997}, 'rating': 4.3, 'userRatingCount': 497, 'displayName': {'text': 'Patriot House Pub', 'languageCode': 'en'}}]}
